## Reading Data

In [43]:
import os
import cv2
from tensorflow.keras.utils import to_categorical

CLASSES = ["flooded", "non-flooded"]
def load_data(data_path):
    data = []
    labels = []
    for c in CLASSES:
        path = os.path.join(data_path, c)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            img_array = cv2.resize(img_array, (227, 227))
            data.append(img_array)
            labels.append(CLASSES.index(c))

    labels = to_categorical(labels)
    return data, labels


# Load data
data, labels = load_data("./Dataset")

## Splitting Data

In [44]:
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

X_train = np.array(X_train)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

X_test = np.array(X_test)
y_test = np.array(y_test)


## Preprocessing

In [45]:
import imgaug.augmenters as iaa
import numpy as np
import random

def data_augmentation(data, labels):
    augmented_images = []
    augmented_labels = []
    seqs = []
    # Define the augmentation sequences
    for i in range(1):
        seq = iaa.Sequential([
            iaa.Fliplr(p=random.uniform(0, 1)),
            iaa.Crop(percent=(0, random.uniform(0, 0.1))),
            iaa.GaussianBlur(sigma=random.uniform(0, 3.0)),
            iaa.AdditiveGaussianNoise(scale=(0, random.uniform(0, 0.1*255))),
            iaa.Multiply((random.uniform(0.5, 1.5), random.uniform(0.5, 1.5))),
            iaa.Affine(
                scale={"x": (random.uniform(0.8, 1.2), random.uniform(0.8, 1.2)), "y": (random.uniform(0.8, 1.2), random.uniform(0.8, 1.2))},
                translate_percent={"x": (random.uniform(-0.2, 0.2), random.uniform(-0.2, 0.2)), "y": (random.uniform(-0.2, 0.2), random.uniform(-0.2, 0.2))},
                rotate=(random.uniform(-45, 45), random.uniform(-45, 45)),
                shear=(random.uniform(-16, 16), random.uniform(-16, 16))
            )
        ], random_order=True)
        seqs.append(seq)

    for i, image in enumerate(data):
        augmented_images.append(image)
        augmented_labels.append(labels[i])
        for seq in seqs:
            augmented_image = seq(image=image)
            augmented_images.append(augmented_image)
            augmented_labels.append(labels[i])
    
    return np.array(augmented_images), np.array(augmented_labels)


In [46]:
# apply augmentation on training data
X_train_aug, y_train = data_augmentation(X_train, y_train)

# apply augmentation on validation data
X_val_aug, y_val = data_augmentation(X_val, y_val)

## Features Extraction

In [47]:
# X_train_aug = np.array([extract_image_features(image) for image in X_train_aug])
# X_val_aug = np.array([extract_image_features(image) for image in X_val_aug])

## Model

### Model Building

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import (
    Conv2D,
    AveragePooling2D,
    Flatten,
    Dense,
    Activation,
    MaxPool2D,
    BatchNormalization,
    Dropout,
)
from tensorflow.keras.utils import to_categorical

In [49]:
# model = Sequential()
# # 1st layer (CONV + pool + batchnorm)
# model.add(
#     Conv2D(
#         filters=96,
#         kernel_size=(11, 11),
#         strides=(4, 4),
#         padding="valid",
#         input_shape=(227, 227, 3),
#     )
# )
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
# model.add(BatchNormalization())
# # 2nd layer (CONV + pool + batchnorm)
# model.add(
#     Conv2D(
#         filters=256,
#         kernel_size=(5, 5),
#         strides=(1, 1),
#         padding="same",
#         kernel_regularizer=l2(0.0005),
#     )
# )
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding="valid"))
# model.add(BatchNormalization())
# # layer 3 (CONV + batchnorm)
# model.add(
#     Conv2D(
#         filters=384,
#         kernel_size=(3, 3),
#         strides=(1, 1),
#         padding="same",
#         kernel_regularizer=l2(0.0005),
#     )
# )
# model.add(Activation("relu"))
# model.add(BatchNormalization())
# # layer 4 (CONV + batchnorm)
# model.add(
#     Conv2D(
#         filters=384,
#         kernel_size=(3, 3),
#         strides=(1, 1),
#         padding="same",
#         kernel_regularizer=l2(0.0005),
#     )
# )
# model.add(Activation("relu"))
# model.add(BatchNormalization())
# # layer 5 (CONV + batchnorm)
# model.add(
#     Conv2D(
#         filters=256,
#         kernel_size=(3, 3),
#         strides=(1, 1),
#         padding="same",
#         kernel_regularizer=l2(0.0005),
#     )
# )
# model.add(Activation("relu"))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding="valid"))
# model.add(Flatten())
# # layer 6 (Dense layer + dropout)
# model.add(Dense(units=4096, activation="relu"))
# model.add(Dropout(0.5))
# # layer 7 (Dense layers)
# model.add(Dense(units=4096, activation="relu"))
# model.add(Dropout(0.5))
# # layer 8 (softmax output layer)
# model.add(Dense(units=2, activation="softmax"))


In [50]:
model = Sequential()
# 1st layer (CONV + pool + batchnorm)
model.add(
    Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding="same",
        input_shape=(227, 227, 3),
    )
)
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())
# 2nd layer (CONV + pool + batchnorm)
model.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_regularizer=l2(0.0005),
    )
)
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2), padding="valid"))
model.add(BatchNormalization())
# layer 3 (CONV + batchnorm)
model.add(
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_regularizer=l2(0.0005),
    )
)
model.add(Activation("relu"))
model.add(BatchNormalization())
# layer 4 (CONV + batchnorm)
model.add(
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_regularizer=l2(0.0005),
    )
)
model.add(Activation("relu"))
model.add(BatchNormalization())
# layer 5 (CONV + batchnorm)
model.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_regularizer=l2(0.0005),
    )
)
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), padding="valid"))
model.add(Flatten())
# layer 6 (Dense layer + dropout)
model.add(Dense(units=1024, activation="relu"))
model.add(Dropout(0.5))
# layer 7 (Dense layers)
model.add(Dense(units=1024, activation="relu"))
model.add(Dropout(0.5))
# layer 8 (softmax output layer)
model.add(Dense(units=2, activation="softmax"))

### Model Training

In [51]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

history = model.fit(
    X_train_aug, y_train,
    batch_size=64,
    epochs=50,
    validation_data=(X_val_aug, y_val)
)

Epoch 1/50
21/21 [==============================] - 22s 1s/step - loss: 4.4940 - accuracy: 0.6621 - val_loss: 58.9291 - val_accuracy: 0.4703
Epoch 2/50
21/21 [==============================] - 22s 1s/step - loss: 2.4148 - accuracy: 0.7609 - val_loss: 21.9336 - val_accuracy: 0.4703
Epoch 3/50
21/21 [==============================] - 21s 1s/step - loss: 1.4445 - accuracy: 0.7888 - val_loss: 5.4349 - val_accuracy: 0.4703
Epoch 4/50
21/21 [==============================] - 19s 910ms/step - loss: 1.2851 - accuracy: 0.8017 - val_loss: 3.3825 - val_accuracy: 0.4649
Epoch 5/50
21/21 [==============================] - 19s 897ms/step - loss: 1.0168 - accuracy: 0.8469 - val_loss: 2.3076 - val_accuracy: 0.5135
Epoch 6/50
21/21 [==============================] - 21s 1s/step - loss: 0.7028 - accuracy: 0.8741 - val_loss: 0.9528 - val_accuracy: 0.7027
Epoch 7/50
21/21 [==============================] - 20s 955ms/step - loss: 0.6585 - accuracy: 0.8771 - val_loss: 1.0622 - val_accuracy: 0.6622
Epoch 8/5

## Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_val, y_pred)
cm_df = pd.DataFrame(cm, index=CATEGORIES, columns=CATEGORIES)
sns.heatmap(cm_df, annot=True, cmap="Blues", fmt="d")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Plot f1 score
f1 = f1_score(y_val, y_pred, average=None)
f1_df = pd.DataFrame(f1, index=CATEGORIES, columns=["F1 Score"])
f1_df.plot(kind="bar", ylim=(0, 1))
plt.title("F1 Score")
plt.show()

# describe history for accuracy
plt.plot(history.history["accuracy"])

NameError: name 'categories' is not defined

### Save Model

In [ ]:
# model.save("model.h5")

### load model

In [ ]:
# load model
from tensorflow.keras.models import load_model

# model = load_model("model.h5")

### Evaluate On Test Data

In [ ]:
# test the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("Test accuracy:", test_acc)

ValueError: in user code:

    File "/home/ahmed/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1525, in test_function  *
        return step_function(self, iterator)
    File "/home/ahmed/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1514, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ahmed/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1507, in run_step  **
        outputs = model.test_step(data)
    File "/home/ahmed/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1473, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/ahmed/.local/lib/python3.10/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/home/ahmed/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_we